# Capítulo 2

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
.builder()
.appName("Scala")
.master("local")
.enableHiveSupport()
.getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4eda30d


    a. Descargar el Quijote (https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79) y aplicar no solo count (para obtener el número de líneas) y show sino probar distintas sobrecargas del método show (con y sin truncate, indicando/sin indicar num de filas, etc) así como también los métodos head, take, first (diferencias entre estos 3?)

In [8]:
// Se puede leer como CSV pero vamos a leerlo como archivo de texto porque es... un archivo de texto.

val QuijoteDF = spark.read.text("el_quijote.txt")

QuijoteDF: org.apache.spark.sql.DataFrame = [value: string]


In [11]:
QuijoteDF.show(5,false)

+----------------------------------------------------------------------------------------------+
|value                                                                                         |
+----------------------------------------------------------------------------------------------+
|DON QUIJOTE DE LA MANCHA                                                                      |
|Miguel de Cervantes Saavedra                                                                  |
|                                                                                              |
|PRIMERA PARTE                                                                                 |
|CAPI?TULO 1: Que trata de la condicio?n y ejercicio del famoso hidalgo D. Quijote de la Mancha|
+----------------------------------------------------------------------------------------------+
only showing top 5 rows



In [14]:
QuijoteDF.count()

res5: Long = 2186


In [22]:
QuijoteDF.take(1)

res13: Array[org.apache.spark.sql.Row] = Array([DON QUIJOTE DE LA MANCHA])


In [21]:
QuijoteDF.head(5)

res12: Array[org.apache.spark.sql.Row] = Array([DON QUIJOTE DE LA MANCHA], [Miguel de Cervantes Saavedra], [], [PRIMERA PARTE], [CAPI?TULO 1: Que trata de la condicio?n y ejercicio del famoso hidalgo D. Quijote de la Mancha])


In [17]:
QuijoteDF.first()

res8: org.apache.spark.sql.Row = [DON QUIJOTE DE LA MANCHA]


Take tiene que recibir un número de líneas si o si, mientras que head puede no recibir ningún número y devolver la primera línea. First siempre devuelve la primera línea. Además, si head no recibe parámetro devuelve una fila, mientras que take(1) devuelve un array que contiene una fila.

    b. Del ejercicio de MnM aplicar:

In [3]:
// Leo el dataset de MnM

// get the M&M data set file name
val mnmFile = "C:/Users/rafael.gomez/learning/chapter2/scala/data/mnm_dataset.csv"
// read the file into a Spark DataFrame
val mnmDF = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(mnmFile)
// display DataFrame
mnmDF.show(5, false)

+-----+------+-----+
|State|Color |Count|
+-----+------+-----+
|TX   |Red   |20   |
|NV   |Blue  |66   |
|CO   |Blue  |79   |
|OR   |Blue  |71   |
|WA   |Yellow|93   |
+-----+------+-----+
only showing top 5 rows



mnmFile: String = C:/Users/rafael.gomez/learning/chapter2/scala/data/mnm_dataset.csv
mnmDF: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]


        i. Otras operaciones de agregación como el Max con otro tipo de ordenamiento (descendiente).

In [42]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._


In [70]:
val caCountMnNDF = mnmDF.select("*")
  .groupBy("State", "Color")
  .max("Count")
  .orderBy(desc("max(Count)"))

// show the resulting aggregation for California
caCountMnNDF.show(10)

+-----+------+----------+
|State| Color|max(Count)|
+-----+------+----------+
|   WA|   Red|       100|
|   NM| Green|       100|
|   NV|   Red|       100|
|   NV| Brown|       100|
|   CA|  Blue|       100|
|   WA|Orange|       100|
|   UT|  Blue|       100|
|   WY| Green|       100|
|   AZ|Orange|       100|
|   NV|Orange|       100|
+-----+------+----------+
only showing top 10 rows



caCountMnNDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]


        ii. hacer un ejercicio como el “where” de CA que aparece en el libro pero indicando más opciones de estados (p.e. NV, TX, CA, CO).

In [71]:
val caCountMnNDF = mnmDF.select("*")
  .where(col("State") === "CA" || col("State") === "NV")
  .groupBy("State", "Color")
  .sum("Count")
  .orderBy(desc("sum(Count)"))

// show the resulting aggregation for California
caCountMnNDF.show(10)

+-----+------+----------+
|State| Color|sum(Count)|
+-----+------+----------+
|   CA|Yellow|    100956|
|   CA| Brown|     95762|
|   NV|Orange|     93929|
|   CA| Green|     93505|
|   NV| Brown|     92478|
|   CA|   Red|     91527|
|   NV|Yellow|     91390|
|   NV| Green|     91331|
|   CA|Orange|     90311|
|   NV|  Blue|     90003|
+-----+------+----------+
only showing top 10 rows



caCountMnNDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]


        iii. Hacer un ejercicio donde se calculen en una misma operación el Max, Min, Avg, Count. Revisar el API (documentación) donde encontrarán este ejemplo:
        ds.agg(max($"age"), avg($"salary")) ds.groupBy().agg(max($"age"), avg($"salary"))
        NOTA: $ es un alias de col()

In [76]:
mnmDF.select(max("Count"),min("Count"),avg("Count"),count("Count")).show()

+----------+----------+-----------------+------------+
|max(Count)|min(Count)|       avg(Count)|count(Count)|
+----------+----------+-----------------+------------+
|       100|        10|55.00090000900009|       99999|
+----------+----------+-----------------+------------+



In [88]:
mnmDF.agg(max($"Count"),min($"Count"),avg($"Count"),count($"Count")).show()

+----------+----------+-----------------+------------+
|max(Count)|min(Count)|       avg(Count)|count(Count)|
+----------+----------+-----------------+------------+
|       100|        10|55.00090000900009|       99999|
+----------+----------+-----------------+------------+



        iv. Hacer también ejercicios en SQL creando tmpView

CreateOrReplaceTempView will create a temporary view of the table on memory it is not persistent at this moment but you can run SQL query on top of that. if you want to save it you can either persist or use saveAsTable to save.

In [89]:
mnmDF.createOrReplaceTempView("mnmTemp")

In [97]:
spark.sql(""" SELECT * from mnmTemp where State=="CA" or Color="Yellow" """).show(5,false)

+-----+------+-----+
|State|Color |Count|
+-----+------+-----+
|WA   |Yellow|93   |
|CA   |Yellow|53   |
|WA   |Yellow|20   |
|NM   |Yellow|15   |
|WY   |Yellow|48   |
+-----+------+-----+
only showing top 5 rows



# Capítulo 3 

## Realizar todos los ejercicios propuestos de libro

In [102]:
// Leer el dataset de fire_ts

import org.apache.spark.sql.types._

val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
 StructField("UnitID", StringType, true),
 StructField("IncidentNumber", IntegerType, true),
 StructField("CallType", StringType, true),
 StructField("CallDate", StringType, true),
 StructField("WatchDate", StringType, true),
 StructField("CallFinalDisposition", StringType, true),
 StructField("AvailableDtTm", StringType, true),
 StructField("Address", StringType, true), 
 StructField("City", StringType, true), 
 StructField("Zipcode", IntegerType, true), 
 StructField("Battalion", StringType, true), 
 StructField("StationArea", StringType, true), 
 StructField("Box", StringType, true), 
 StructField("OriginalPriority", StringType, true), 
 StructField("Priority", StringType, true), 
 StructField("FinalPriority", IntegerType, true), 
 StructField("ALSUnit", BooleanType, true), 
 StructField("CallTypeGroup", StringType, true),
 StructField("NumAlarms", IntegerType, true),
 StructField("UnitType", StringType, true),
 StructField("UnitSequenceInCallDispatch", IntegerType, true),
 StructField("FirePreventionDistrict", StringType, true),
 StructField("SupervisorDistrict", StringType, true),
 StructField("Neighborhood", StringType, true),
 StructField("Location", StringType, true),
 StructField("RowID", StringType, true),
 StructField("Delay", FloatType, true)))

// Read the file using the CSV DataFrameReader
val sfFireFile="C:/Users/rafael.gomez/learning/chapter3/data/sf-fire-calls.csv"
val fireDF = spark.read.schema(fireSchema)
 .option("header", "true")
 .csv(sfFireFile)

import org.apache.spark.sql.types._
fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), ...


In [111]:
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
newFireDF
 .select("ResponseDelayedinMins")
 .where($"ResponseDelayedinMins" > 5)
 .show(5, false)

+---------------------+
|ResponseDelayedinMins|
+---------------------+
|5.35                 |
|6.25                 |
|5.2                  |
|5.6                  |
|7.25                 |
+---------------------+
only showing top 5 rows



newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


In [112]:
// Transforma las fechas de fire_DF de String a fecha. (y renombra las columnas)

val fireTsDF = newFireDF
 .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy"))
 .drop("CallDate")
 .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy"))
 .drop("WatchDate")
 .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"),
 "MM/dd/yyyy hh:mm:ss a"))
 .drop("AvailableDtTm")
// Select the converted columns
fireTsDF
 .select("IncidentDate", "OnWatchDate", "AvailableDtTS")
 .show(5, false)


+-------------------+-------------------+-------------------+
|IncidentDate       |OnWatchDate        |AvailableDtTS      |
+-------------------+-------------------+-------------------+
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:44|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 03:01:18|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:39:50|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 04:16:46|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 06:01:58|
+-------------------+-------------------+-------------------+
only showing top 5 rows



fireTsDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


In [120]:
// Ahora que tenemos la fecha en un formado reconocido por Spark podemos acceder directamente al día, año o mes.

fireTsDF
 .select(year($"IncidentDate"))
 .distinct()
 .orderBy(year($"IncidentDate"))
 .show()

+------------------+
|year(IncidentDate)|
+------------------+
|              2000|
|              2001|
|              2002|
|              2003|
|              2004|
|              2005|
|              2006|
|              2007|
|              2008|
|              2009|
|              2010|
|              2011|
|              2012|
|              2013|
|              2014|
|              2015|
|              2016|
|              2017|
|              2018|
+------------------+



In [122]:
// What where all the different types of fire calls in 2018?
(fireTsDF.select($"CallType",year($"IncidentDate"))
.distinct()
.where(year($"IncidentDate") === 2018)
.show())

+--------------------+------------------+
|            CallType|year(IncidentDate)|
+--------------------+------------------+
|   Electrical Hazard|              2018|
|          Fuel Spill|              2018|
|        Water Rescue|              2018|
|    Medical Incident|              2018|
|        Vehicle Fire|              2018|
|  Suspicious Package|              2018|
|               Other|              2018|
|              HazMat|              2018|
|Gas Leak (Natural...|              2018|
|       Assist Police|              2018|
|Smoke Investigati...|              2018|
|Elevator / Escala...|              2018|
|Odor (Strange / U...|              2018|
|      Structure Fire|              2018|
|              Alarms|              2018|
|   Traffic Collision|              2018|
|Citizen Assist / ...|              2018|
|Train / Rail Inci...|              2018|
|        Outside Fire|              2018|
|           Explosion|              2018|
+--------------------+------------

In [132]:
// What months within the year 2018 saw the highest number of fire calls?

(fireTsDF
 .where(year($"IncidentDate") === 2018)
 .groupBy(month($"IncidentDate"))
 .count()
 .orderBy(desc("count"))
 .show(5,false))

+-------------------+-----+
|month(IncidentDate)|count|
+-------------------+-----+
|10                 |1068 |
|5                  |1047 |
|3                  |1029 |
|8                  |1021 |
|1                  |1007 |
+-------------------+-----+
only showing top 5 rows



In [165]:
// Which neighborhood in San Francisco generated the most fire calls in 2018?

(fireTsDF
 .select($"Neighborhood")
 .where($"City" === "San Francisco" && year($"IncidentDate") === 2018)
 .groupBy($"Neighborhood")
 .count()
 .orderBy(desc("count"))
 .show(5))

+--------------------+-----+
|        Neighborhood|count|
+--------------------+-----+
|          Tenderloin| 1393|
|     South of Market| 1052|
|             Mission|  911|
|Financial Distric...|  764|
|Bayview Hunters P...|  513|
+--------------------+-----+
only showing top 5 rows



In [189]:
// Which neighborhoods had the worst response times to fire calls in 2018?

// No sé si pide qué barrios tienen peores tiempos así en general o cuales tienen el peor tiempo absoluto
// Así que cojo el peor tiempo promedio.

(fireTsDF
 .select($"Neighborhood",$"ResponseDelayedinMins")
 .where(year($"IncidentDate") === 2018)
 .groupBy($"Neighborhood")
 .agg(avg($"ResponseDelayedinMins").alias("promedioTiempo"))
 .orderBy(desc("promedioTiempo"))
 .show(5))

+--------------------+-----------------+
|        Neighborhood|   promedioTiempo|
+--------------------+-----------------+
|           Chinatown|6.190314101143033|
|            Presidio|5.829227011272873|
|     Treasure Island|5.453703684111436|
|        McLaren Park| 4.74404764175415|
|Bayview Hunters P...|4.620561962212182|
+--------------------+-----------------+
only showing top 5 rows



In [202]:
// Which week in the year in 2018 had the most fire calls?

(fireTsDF
 .select(weekofyear($"IncidentDate").alias("Semana"))
 .where(year($"IncidentDate") === 2018)
 .groupBy($"Semana")
 .count()
 .orderBy(desc("count"))
 .show(5))

+------+-----+
|Semana|count|
+------+-----+
|    22|  259|
|    40|  255|
|    43|  250|
|    25|  249|
|     1|  246|
+------+-----+
only showing top 5 rows



In [ ]:
// Is there a correlation between neighborhood, zip code, and number of fire calls?

In [ ]:
// How can we use Parquet files or SQL tables to store this data and read it back?

## Leer el CSV del ejemplo del cap2 y obtener la estructura del schema dado por defecto

In [203]:
// Imagino que el CSV del que estamos hablando es el de los MnM

mnmDF.printSchema()

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)



## Cuando se define un schema al definir un campo por ejemplo StructField('Delay', FloatType(), True) ¿qué significa el último parámetro Boolean?

Indica si el campo puede ser nulo.

## Dataset vs DataFrame (Scala). ¿En qué se diferencian a nivel de código?

En el JVM, las filas en un dataframe no tienen tipo pero en un dataset tienen un tipo estático.

## Utilizando el mismo ejemplo utilizado en el capítulo para guardar en parquet y guardar los datos en los formatos:

In [4]:
// El código en cuestión para guardar en parquet

mnmDF.write.format("parquet")
 .mode("overwrite")
 .option("compression", "snappy")
 .save("/tmp/data/parquet/df_parquet")


### JSON

In [7]:
mnmDF.write.format("json")
 .mode("overwrite")
 .save("/tmp/data/json/df_json")


### CSV

In [9]:
mnmDF.write.format("csv").mode("overwrite").save("/tmp/data/csv/df_csv")

### AVRO

In [11]:
mnmDF.write
 .format("avro")
 .mode("overwrite")
 .save("/tmp/data/avro/df_avro")

# Capítulo 4

## GlobalTempView vs TempView

EL tiempo de vida de una TempView está asociado al tiempo de vida de la sparkSession actual, mientras que la de GlobalTempView está asociada a la propia aplicación de Spark

## Leer los AVRO, Parquet, JSON y CSV escritos en el cap3

In [12]:
// Parquet

val file = """/tmp/data/parquet/df_parquet"""
val mnmdfparq = spark.read.format("parquet").load(file)

mnmdfparq.show(5)

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
+-----+------+-----+
only showing top 5 rows



file: String = /tmp/data/parquet/df_parquet
mnmdfparq: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]


In [13]:
// Json

val file = "/tmp/data/json/df_json*"
val mnmdfjson = spark.read.format("json").load(file)

mnmdfjson.show(5)

+------+-----+-----+
| Color|Count|State|
+------+-----+-----+
|   Red|   20|   TX|
|  Blue|   66|   NV|
|  Blue|   79|   CO|
|  Blue|   71|   OR|
|Yellow|   93|   WA|
+------+-----+-----+
only showing top 5 rows



file: String = /tmp/data/json/df_json*
mnmdfjson: org.apache.spark.sql.DataFrame = [Color: string, Count: bigint ... 1 more field]


In [15]:
// CSV

val file = "/tmp/data/csv/df_csv/*"

val mnmdfcsv= spark.read.format("csv").load(file)

mnmdfcsv.show(5)


+---+------+---+
|_c0|   _c1|_c2|
+---+------+---+
| TX|   Red| 20|
| NV|  Blue| 66|
| CO|  Blue| 79|
| OR|  Blue| 71|
| WA|Yellow| 93|
+---+------+---+
only showing top 5 rows



file: String = /tmp/data/csv/df_csv/*
mnmdfcsv: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 1 more field]


In [16]:
// AVRO

val mnmdfavro = spark.read.format("avro")
.load("/tmp/data/avro/df_avro/*")

mnmdfavro.show(5)

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
+-----+------+-----+
only showing top 5 rows



mnmdfavro: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]


# Capítulo 5 

## Instalar MySQL, descargar driver y cargar datos de BBDD de empleados 
https://dev.mysql.com/doc/employee/en/

### Cargar con spark datos de empleados y departamentos

In [18]:
// mysql -u root -p < employees.sql (en CMD)

In [17]:
val employees = spark
.read
.format("jdbc")
.option("url", "jdbc:mysql://localhost:3306/employees")
.option("driver", "com.mysql.jdbc.Driver")
.option("dbtable", "employees")
.option("user", "root")
.option("password", "root")
.load()

java.lang.ClassNotFoundException:  com.mysql.jdbc.Driver